## Lisamaterjal: Märgenduste visualiseerimine I
---

Selles materjalis tutvustame, kuidas EstNLTK andmestruktuure lihtsalt ja ilusalt kuvada ning mis moel välja tuua meid tekstis huvitavaid kihte ja nende märgendusi.

### Jupyter Notebook-i funktsioon `display()`

Jupyter Notebook-i on sisse ehitatud funktsioon [`display()`](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.display), mille abil on võimalik erinevat tüüpi sisu (nt HTML lehekülje lähtekoodi, JSON andmeid, pildifaile) kuvada.
Seda funktsioon saab ka kasutada EstNLTK andmestruktuuride **järjepidevaks** kuvamiseks.

Tavaliselt kuvab Jupyter Notebook vaid koodilõigu viimase objekti:

In [1]:
# Loome näidisteksti
from estnltk import Text
text = Text('Tere!').tag_layer()
# Objektid järjestikku tavaliselt
text.morph_analysis[0]
text.morph_analysis[1]

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
!,!,!,!,['!'],,,,Z


aga `display()` abil saame järjepidevalt kuvada kõik:

In [2]:
# Objektid järjestikku display() abil
display( text.morph_analysis[0] )
display( text.morph_analysis[1] )

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
Tere,Tere,tere,tere,['tere'],0,,,I


text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
!,!,!,!,['!'],,,,Z


Seega on `display()` kasulik näiteks juhtudel, kui on tarvis tsükli sees igal sammul midagi kuvada.

Mõnevõrra tehnilisem `display()` kasutusjuhtum: kui on tarvis ise HTML lähtekoodi kokku panna ja kuvada, siis saab seda teha `HTML()`-i ja `display()`-id kombineerides, nt:

In [3]:
from IPython.display import HTML

display(HTML('<br>See on <b>paksus kirjas</b> ja <i>kaldkirjas</i>!'))

HTML märgendustest on detailsemalt juttu 8. praktikumis.

EstNLTK märgendusi pole aga vaja ise programmaatiliselt HTML-iks teha: on olemas meetodid ja klassid, mis teevad selle töö meie eest ära. 
Nendest tulebki järgnevalt juttu.

---

### Kihi meetod `display()`

Mõnikord on tarvis saada mingi kihi märgendustest kiire ülevaade. 
Selleks oleks tore kuvada tekst selliselt, et märgendused on spetsiaalselt esile toodud.
EstNLTK-s saab selleks kasutada kihi meetodit `display()`:

In [4]:
# Loome teksti koos lausete ja morf märgendusega
from estnltk import Text
text = Text("1000 ligikaudu 10 000-st Bayeris 1913. aastal töötanud inimesest töötas väljaspool Saksamaad . ").tag_layer()
# Kuvame kõik sõnad värviliselt
text.words.display()

1000 ligikaudu 10 000-st Bayeris 1913. aastal töötanud inimesest töötas väljaspool Saksamaad .

In [5]:
# Kuvame värviliselt kõik compound_tokens kihi elemendid
text.compound_tokens.display()

1000 ligikaudu 10 000-st Bayeris 1913. aastal töötanud inimesest töötas väljaspool Saksamaad .

_Kujunduse stiili määramine._
Meetodile `display()` on võimalik kaasa anda parameeter `mapping_dict`, milles on kujunduse stiili kirjeldav sõnastik.
Sõnastikus on [CSS elementidele](https://www.htmldog.com/references/css/properties/) (nt "background", "font-weight" või "font-family") seatud vastavusse funktsioon, mis saab sisendiks ühe argumendi (list, mis sisaldab tekstisisu ja sellele vastavaid `Annotation` objekte) ning otsustab argumendi väärtuse põhjal, milline peaks olema CSS elemendi väärtus. 
Kõige lihtsamalt juhul võime kasutada `lambda` funktsioone, mis tagastavad alati kindla väärtuse, nt:

In [6]:
display(text.compound_tokens)

Layer(name='compound_tokens', attributes=('type', 'normalized'), spans=SL[EnvelopingSpan(['10', '000', '-', 'st'], [{'type': ['numeric', 'hyphenation', 'case_ending'], 'normalized': '10000-st'}]),
EnvelopingSpan(['1913', '.'], [{'type': ['numeric'], 'normalized': '1913.'}])])

In [7]:
# Defineerime CSS stiili elemendid:
mapping_dict = {"background":  (lambda x:'lime'),     # taustavärv: laimroheline
                "font-weight": (lambda x:'bold'),     # paks kiri
                "font-family": (lambda x:'Courier')}  # kirjatüüp: Courier

# Rakendame uut stiili liitsõnedega teksti kujundamisel
text.compound_tokens.display( mapping_dict = mapping_dict )

1000 ligikaudu 10 000-st Bayeris 1913. aastal töötanud inimesest töötas väljaspool Saksamaad .

Sõnastiku `mapping_dict` sees olevad funktsioonid saavad argumendiks listi, mille esimene element on parasjagu kujundatav tekstisisu (sõne) ning teine element on list tekstisisule vastavatest `Annotation` objektidest.
Mitu elementi `Annotation` objektide listis on, sõltub sellest, kas tegemist on ühese või mitmese märgendusega; üheste kihtide puhul on seal alati üks element. 
Seda teadmist ära kasutades võime defineerida keerulisema `mapping_dict`-i, kus kujunduselementide väärtused sõltuvad `Annotation` objektide sisust, nt tahame teisiti märgendada käändelõpuga mitmeosalisi sõnu:

In [8]:
# Defineerime CSS stiili, kus taustavärv sõltub liitsõne tüübist:
#   --> kui tüüp on 'case_ending', siis taust on roosakas, muudel juhtudel aga laimroheline;
mapping_dict = {"background":      lambda x: 'pink' if 'case_ending' in x[1][0]['type'] else 'lime',
                "font-weight":     lambda x: 'bold',      # paksus kirjas
                "letter-spacing":  lambda x : '3px' }     # sõrendusega tekst
# Rakendame uut stiili liitsõnede värvimisel
text.compound_tokens.display( mapping_dict = mapping_dict )

1000 ligikaudu 10 000-st Bayeris 1913. aastal töötanud inimesest töötas väljaspool Saksamaad .

Täiendavad märkused:

 * Loomulikult ei pea piirduma `lambda` funktsioonide kasutamisega: kui on tarvis luua keerukamat loogikat, mis hõlmab mitut koodirida, siis võib defineerida eraldiseisva funktsiooni ja kasutada seda `mapping_dict`-i sees:
 
            def my_background_decorator( span_list ):
                ...
                ...
                return ...
            
            mapping_dict = {"background":  my_background_decorator }

 * Milliseid tekstivormingu võimalusi CSS veel sisaldab, selle kohta vaata lähemalt [siit](https://www.w3schools.com/css/css_text.asp);

 * Värvide valimisel tasub ilmselt olla nüansirikkam: ei tasu piirduda värvidega, mis on võtmesõnadega defineeritud (_a la_ 'lime', 'pink', 'red', 'blue'), vaid tasub uurida [värvide kirjeldamist](https://www.htmldog.com/references/css/values/color/) `rgb()` funktsiooni või kuueteistkümnendsüsteemi HEX koodi järgi. Märksõnaks on [_HTML color picker_](https://www.google.com/search?q=html+color+picker).